In [57]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [58]:
## Loading the dataset

data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [59]:
#splti the data into training, validation and test sets

train, test = train_test_split(data,test_size = 0.25, random_state = 42)

In [60]:
x_train = train.drop(["quality"],axis=1).values
y_train = train[["quality"]].values.ravel()

#Validation data
x_test = test.drop(['quality'],axis=1).values
y_test = test[['quality']].values.ravel()

#Splitting the data into training and validation data further

train_x,valid_x, train_y,valid_y= train_test_split(x_train,y_train, test_size = 0.20, random_state = 42)

signature = infer_signature(x_train,y_train)

In [61]:
#ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ##Define model architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]

    )

    ##Compiling the mode
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],momentum=params["momentum"]
    ),
    loss = "mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
                 )

    ##Train the ann model with lr and momentum parameteres with mlflow tracking


    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                epochs=epochs,
                batch_size=64)

        ##Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ##Logging the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        #log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_rmse, "status":STATUS_OK, "model":model}

In [62]:
def objective(params):
    #MLFLOW will track the parameters and results for each run here
    result = train_model(
        params,
        epochs=3,
        train_x = x_train,
        train_y = y_train,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = x_test,
        test_y = y_test,
    )
    return result


In [63]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.loguniform("momentum",np.log(1e-5),np.log(1.0)) # Corrected momentum space
}

In [66]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    ##Conducting the hyperparameter search using hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    #Fetching the details of the best run
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]

    ##Log the best model parameters, loss and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model", signature=signature)

    #print out the best parameters and corresponding loss
    print(f"Best parameters:{best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 22s 399ms/step - loss: 41.5611 - root_mean_squared_error: 6.4468
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 16.8429 - root_mean_squared_error: 3.9794 - val_loss: 1.9398 - val_root_mean_squared_error: 1.3928

Epoch 2/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.0871 - root_mean_squared_error: 1.4447
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - loss: 1.7545 - root_mean_squared_error: 1.3237 - val_loss: 1.3937 - val_root_mean_squared_error: 1.1805

Epoch 3/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.5178 - root_mean_squared_error: 1.2320
36/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2893 - root_mean_squared_error: 1.1353
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2710 - root_mean_squared_error: 1.1272 - val_loss: 1.1

2025/08/28 08:20:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run funny-shark-624 at: http://127.0.0.1:5000/#/experiments/272991609569955364/runs/0b6f99644a284678a0138c1161593f47

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/272991609569955364   

Epoch 1/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 16s 284ms/step - loss: 39.1179 - root_mean_squared_error: 6.2544
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.4871 - root_mean_squared_error: 6.2838 - val_loss: 37.5513 - val_root_mean_squared_error: 6.1279

Epoch 2/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 38.3866 - root_mean_squared_error: 6.1957
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - loss: 36.8873 - root_mean_squared_error: 6.0734 - val_loss: 35.1891 - val_root_mean_squared_error: 5.9320

Epoch 3/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 35.5442 - root_mean_sq

2025/08/28 08:21:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run unleashed-hare-58 at: http://127.0.0.1:5000/#/experiments/272991609569955364/runs/ca90851bfc6d4633b0f8628e6ed4a427

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/272991609569955364   

Epoch 1/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 16s 283ms/step - loss: 25.4616 - root_mean_squared_error: 5.0460
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.0430 - root_mean_squared_error: 5.0037 - val_loss: 20.6493 - val_root_mean_squared_error: 4.5441

Epoch 2/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19.8802 - root_mean_squared_error: 4.4587
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - loss: 19.0138 - root_mean_squared_error: 4.3600 - val_loss: 15.7396 - val_root_mean_squared_error: 3.9673

Epoch 3/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 14.6627 - root_mean_

...

🏃 View run silent-stoat-993 at: http://127.0.0.1:5000/#/experiments/272991609569955364/runs/296a63d5fca0488bb67a0977e57b135d

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/272991609569955364   

Epoch 1/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 14s 250ms/step - loss: 25.8786 - root_mean_squared_error: 5.0871
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.2636 - root_mean_squared_error: 4.2427 - val_loss: 4.5565 - val_root_mean_squared_error: 2.1346

Epoch 2/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4.1882 - root_mean_squared_error: 2.0465
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - loss: 3.7503 - root_mean_squared_error: 1.9342 - val_loss: 2.5636 - val_root_mean_squared_error: 1.6011

Epoch 3/3                                                                       

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2.9609 - root_mean_square

2025/08/28 08:21:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



🏃 View run crawling-loon-780 at: http://127.0.0.1:5000/#/experiments/272991609569955364/runs/dea626524b964e73ad44bfa9f1fd1225

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/272991609569955364   

100%|████████████| 4/4 [00:35<00:00,  8.78s/trial, best loss: 1.066747784614563]

2025/08/28 08:21:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters:{'lr': np.float64(0.005784172587984911), 'momentum': np.float64(0.0008709498914678489)}
Best eval rmse: 1.066747784614563
🏃 View run silent-perch-880 at: http://127.0.0.1:5000/#/experiments/272991609569955364/runs/d76adc24d9334601b1f26bdd6057bc05
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/272991609569955364
